In [16]:
from Data_Input import Input_Data, leaderboard_data
from Data_Processing import Basic_FE
from Data_Splitting import train_test, train_test_val
import numpy as np
import pandas as pd
import importlib


#If things aren't working, restart kernel to force it to update imports
#because python is dumb!

In [17]:
# Inputs and processes data
data = Input_Data()
p_data = Basic_FE(data)
#processed_data.info()


In [18]:
#Splits and resamples data
#split = train_test_val(p_data, 0.2,0.1)
split = train_test(p_data, 0.2)
split.resample()

In [19]:
#Fits and tests data

# Basic Idea
Make assumptions (feature engineering, what else..?)
Run models (test assumptions)

In [20]:
# Do different ML systems in separate files
# Make a submission file as well
# Bootstrapping option
# Make ML model class to interface with rest of stuff




#Basic structure:

#X_test Y_test X_train_ Y_train

#import model1 from model1
#import model2 from model2
#
#models = [model1,model2,]
#
# for model in models:
# model1(processed_data) (prints data)

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix

# Create AdaBoostClassifier with DecisionTreeClassifier as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)  # Decision Stump
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=150, random_state=42)
adaboost.fit(split.trainx, split.trainy, )


# Predict on the test data
y_pred = adaboost.predict(split.testx)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

NameError: name 'y_test' is not defined

In [24]:
# Calculate the accuracy
accuracy = accuracy_score(split.testy, y_pred)
# confusion matrix
print(confusion_matrix(split.testy, y_pred > .5))
print(f"Accuracy: {accuracy:.2f}")
print(
    f"Balanced Accuracy: {balanced_accuracy_score(split.testy, y_pred > .5)}"
)

[[26947  9949]
 [   53   218]]
Accuracy: 0.73
Balanced Accuracy: 0.7673891088704903


In [11]:

lb_data = leaderboard_data()
#Adjust data

In [12]:
list(lb_data)

['ID',
 'axonal_coor_x',
 'axonal_coor_y',
 'axonal_coor_z',
 'dendritic_coor_x',
 'dendritic_coor_y',
 'dendritic_coor_z',
 'adp_dist',
 'post_skeletal_distance_to_soma',
 'pre_skeletal_distance_to_soma',
 'pre_oracle',
 'pre_test_score',
 'pre_rf_x',
 'pre_rf_y',
 'post_oracle',
 'post_test_score',
 'post_rf_x',
 'post_rf_y',
 'compartment',
 'pre_brain_area',
 'post_brain_area',
 'pre_nucleus_x',
 'pre_nucleus_y',
 'pre_nucleus_z',
 'post_nucleus_x',
 'post_nucleus_y',
 'post_nucleus_z',
 'pre_nucleus_id',
 'post_nucleus_id',
 'pre_feature_weights',
 'post_feature_weights',
 'pre_morph_embeddings',
 'post_morph_embeddings']

In [13]:
# Submission

IDs = lb_data["ID"]
lb_data = Basic_FE(lb_data)
# oversample connected neuron pairs for full training data

data = Input_Data()
train_data = Basic_FE(data)
train_data = train_test(train_data)
train_data.resample()

# fit model
adaboost.fit(train_data.trainx, train_data.trainy)

# predict on leaderboard data
#lb_data["pred"] = pipe.predict_proba(lb_data[["fw_similarity", "adp_dist"]])[:, 1]
lb_data["pred"] = adaboost.predict(lb_data)
lb_data["ID"]=IDs

#create a boolean prediction solution
lb_data["connected"] = lb_data["pred"] > .5

#columns should be ID, connected
submission_data = lb_data.filter(['ID','connected'])
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ID         42593 non-null  int64
 1   connected  42593 non-null  bool 
dtypes: bool(1), int64(1)
memory usage: 374.5 KB


In [14]:
#writing csv files
submission_data.to_csv('tbd5.csv',index=False)